# 04. 모듈 개발 : 강화학습 기반 주식투자 시스템 개발 (2) - Agent 모듈 개발

<img src="figures/cap01.jpeg" width=600 />
<img src="figures/cap02.jpeg" width=600 />
<img src="figures/cap03.jpeg" width=600 />
<img src="figures/cap04.jpeg" width=600 />

# 4.2 RLTrader의 구조 

## 4.2.1 모듈 구조

<img src="figures/cap05.jpeg" width=600 />

## 4.2.2 디레토리 구조

In [3]:
%ls ../rltrader/

agent_custom.py         img/                 policy_network_dnn.py
agent.py                LICENSE              policy_network.py
creon.py                main_custom.py       portal.py
data/                   _main_notraining.py  preprocessor.ipynb
data_manager_custom.py  main_notraining.py   README.md
data_manager.py         main.py              settings.py
environment.py          policy_learner.py    visualizer.py


## 4.2.3 에이전트 모듈 개요

Agent 클래스 (Agent)
* 투자자 역할 하는 클래스
* 내부 상태 : 초기 자본금, 현금 잔고, 주식 잔고
* 포트폴리오 가치(PV)
    * PV = 주식 잔고 X 현재 주가 + 현금 잔고


## 4.2.4 환경 모듈 개요

환경 클래스 (Environment)
* 에이전트가 투자할 차트 데이터를 관리함
* 과거 시점부터 가장 최근 시점까지 순차적으로 데이터를 제공
* 학습을 위해 과거로 돌아간 에이전트는, 미래의 차트 데이터는 알 수 없다.


## 4.2.5 정책 신경망 모듈 개요

정책 신경망 클래스(PolicyNetwork)
* 특정 시점의 주식 데이터(sample)가 제공되었을 때
    * 매수 또는
    * 매도
    * 를 결정하는 뇌와 같은 역할
* LSTM으로 구성되어 있고, 매수와 매도 행위에 대해 PV를 높일 수 있을지의 확률을 계산
* RLTrader는 정책 신경망을 통해 주식 잔고를 늘릴지 줄일지를 결정.
    * 주식 데이터 -> 정책 신경망 -> 매수와 매도의 확률이 나오고 그 중 확률이 가장 높은 것을 선택


## 4.2.6 가시화기 모듈 개요

주식 데이터 학습 과정을 직관적으로 파악하기 위해  환경, 에이전트 상태, 정책 신경망 출력 등을 그림 파일로 시각화

## 4.2.7 정책 학습기 모듈 개요

정책 학습기 클래스 
* 앞의 모듈들을 가지고 강화학습을 수행
* RLTrader의 몸체
* 학습 데이터를 가지고 있음
* 정책 신경망을 학습시킴

# 4.4 에이전트 모듈 개발

## 4.4.1 에이전트 모듈의 주요 속성과 함수

## 속성
* STATE_DIM = 2  # 주식 보유 비율, 포트폴리오 가치 비율

##### 매매 수수료 및 세금
* TRADING_CHARGE = 0  # 거래 수수료 미고려 (일반적으로 0.015%)
* TRADING_TAX = 0  # 거래세 미고려 (실제 0.3%)

##### 행동
* ACTION_BUY = 0  # 매수
* ACTION_SELL = 1  # 매도
* ACTION_HOLD = 2  # 홀딩
* ACTIONS = [ACTION_BUY, ACTION_SELL]  # 인공 신경망에서 확률을 구할 행동들
* NUM_ACTIONS = len(ACTIONS)  # 인공 신경망에서 고려할 출력값의 개수

* environment  # 현재 주식 가격을 가져오기 위해 환경 참조

##### 최소 매매 단위, 최대 매매 단위, 지연보상 임계치
* min_trading_unit  # 최소 단일 거래 단위
* max_trading_unit  # 최대 단일 거래 단위
* delayed_reward_threshold  # 지연보상 임계치

##### Agent 클래스의 속성
* initial_balance = 0  # 초기 자본금
* balance = 0  # 현재 현금 잔고
* num_stocks = 0  # 보유 주식 수
* portfolio_value = 0  # balance + num_stocks * {현재 주식 가격}
* base_portfolio_value = 0  # 직전 학습 시점의 PV
* num_buy = 0  # 매수 횟수
* num_sell = 0  # 매도 횟수
* num_hold = 0  # 홀딩 횟수
* immediate_reward = 0  # 즉시 보상

##### Agent 클래스의 상태
* ratio_hold = 0  # 주식 보유 비율
* ratio_portfolio_value = 0  # 포트폴리오 가치 비율

## 함수
* \__init\__ <- environment, min_trading_unit=1, max_trading_unit=2, delayed_reward_threshold=.05
* reset
* set_balance <- balance
* get_states  
* decide_action <- policy_network, sample, epsilon
* validate_action <- action
* decide_trading_unit <- confidence
* act <- action, confidence

In [17]:
import agent as AGT

In [16]:
import importlib as imp 

In [18]:
agent = AGT.Agent()

In [19]:
dir(agent)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

------------------------

In [ ]:
img.reload(AGT)

In [ ]:
agent = AGT.Agent()

In [ ]:
agent.reset()

In [ ]:
agent.set_balance(balance)

In [ ]:
agent.get_states()

In [ ]:
agent.decide_action(policy_network, sample, epsilon)

In [ ]:
agent.validate_action(action)

In [ ]:
agent.decide_trading_unit(confidence)

In [ ]:
agent.act(action, confidence)

## 4.4.2 코드 조각 1 : 에이전트 클래스의 상수 선언 부분

## 4.4.3 코드 조각 2 : 에이전트 클래스의 생성자 부분

## 4.4.3 코드 조각 3 : 에이전트 클래스의 함수 부분

In [ ]:
get/set

In [ ]:
유효성 검사

In [ ]:
매수/매도 단위 결정 함수

In [ ]:
투자 행동 수행 함수(행동 준비)

In [ ]:
투자 행동 수행 함수(매수)

In [ ]:
투자 행동 수행 함수(매도 및 관망)

In [ ]:
투자 행동 수행 함수(포트폴리오 가치 갱신 및 지연 보상 판단)